In [2]:
import numpy as np
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, \
    AveragePooling2D, MaxPool2D, GlobalAveragePooling2D
from keras.models import Model, load_model
from  keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras_applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from resnets_utils import *
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline

import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

Using TensorFlow backend.


In [3]:
def identify_block(X, f, filters, stage, block):
	
	conv_name_base = 'res' + str(stage) + block + 'branch'
	bn_name_base = 'bn' + str(stage) + block + 'branch'
	
	F1, F2, F3 = filters
	
	X_shortcut = X
	
	X = Conv2D(filters=F1, kernel_size=(1,1), strides=(1,1), padding='valid', name=conv_name_base+'2a', 
               kernel_initializer=glorot_uniform(seed=0))(X)
	X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
	X = Activation('relu')(X)
	
	X = Conv2D(filters=F2, kernel_size=(f,f), strides=(1,1), padding='same', name=conv_name_base+'2b', 
	           kernel_initializer=glorot_uniform(seed=0))(X)
	X = BatchNormalization(axis=3, name=bn_name_base+'2b')(X)
	X = Activation('relu')(X)
	
	X = Conv2D(filters=F3, kernel_size=(1,1), strides=(1,1), padding='valid', name=conv_name_base+'2c',
	           kernel_initializer=glorot_uniform(seed=0))(X)
	X = BatchNormalization(axis=3, name=bn_name_base+'2c')(X)
	
	X = Add()([X, X_shortcut])
	X = Activation('relu')(X)
	
	return X

In [4]:
tf.reset_default_graph()

with tf.Session() as test:
	np.random.seed(1)
	A_prev = tf.placeholder('float', [3, 4, 4, 6])
	X = np.random.randn(3,4,4,6)
	A = identify_block(A_prev, f=2, filters=[2,4,6], stage=1, block='a')
	test.run(tf.global_variables_initializer())
	out = test.run([A], feed_dict={A_prev:X, K.learning_phase():0})
	print(str(out[0][1][1][0]))

Instructions for updating:
Colocations handled automatically by placer.
[ 0.1971682 -0.         1.3561226  2.1713073 -0.         1.3324987]


In [5]:
def convolutional_block(X, f, filters, stage, block, s=2):
	
	conv_name_base = 'res' + str(stage) + block + '_branch'
	bn_name_base = 'bn' + str(stage) + block + '_branch'
	
	F1, F2, F3 = filters
	
	X_shortcut = X
	########## MAIN PATH ###############
	
	X = Conv2D(filters=F1, kernel_size=(1,1), strides=(s,s), name=conv_name_base+'2a', 
	           kernel_initializer=glorot_uniform(seed=0))(X)
	X = BatchNormalization(axis=3, name=bn_name_base+'2a')(X)
	X = Activation('relu')(X)
	
	X = Conv2D(filters=F2, kernel_size=(f,f), strides=(1,1), name=conv_name_base+'2b',
	           kernel_initializer=glorot_uniform(seed=0))(X)
	X = BatchNormalization(axis=3, name=bn_name_base+'2b')(X)
	X = Activation('relu')(X)
	
	X = Conv2D(filters=F3, kernel_size=(1,1), strides=(s,s), name=conv_name_base+'2c',
	           kernel_initializer=glorot_uniform(seed=0))(X)
	X = BatchNormalization(axis=3, name=bn_name_base+'2c')(X)
	
	X_shortcut = Conv2D(filters=F3, kernel_size=(1,1), strides=(s,s), name=conv_name_base+'1',
	                    kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
	X_shortcut = BatchNormalization(axis=3, name=bn_name_base+'1')(X_shortcut)
	
	X = Add()([X, X_shortcut])
	X = Activation('relu')(X)
	
	return X
	

In [6]:
tf.reset_default_graph()

with tf.Session() as test:
	
	np.random.seed(1)
	A_prev = tf.placeholder('float', [3,4,4,6])
	X = np.random.randn(3,4,4,6)
	A = convolutional_block(A_prev, f=2, filters=[2,4,6], stage=1, block='a')
	test.run(tf.global_variables_initializer())
	out = test.run([A], feed_dict={A_prev:X, K.learning_phase():0})
	print(str(out[0][1][1][0]))

[0.50418824 1.3419094  1.1175836  0.25036487 1.1973622  0.6871328 ]


In [11]:
def ResNet50(input_shape=(64,64,3), classes=6):
	
	#Define the input as tensor with shape input_shape
	X_input = Input(input_shape)
	
	#Zero_padding
	X = ZeroPadding2D((3,3))(X_input)
	
	#Stage 1
	X = Conv2D(filters=64, kernel_size=(7,7), strides=(2,2), name='conv1', kernel_initializer=glorot_uniform(seed=0))(X)
	X = BatchNormalization(axis=3, name='bn_conv1')(X)
	X = Activation('relu')(X)
	X = MaxPool2D((3,3), strides=(2,2))(X)
	
	#Stage 2
	X = convolutional_block(X, f=3, filters=[64,64,256], stage=2, block='a', s=1)
	X = identify_block(X, 3, [64,64,256], stage=2, block='b')
	X = identify_block(X, 3, [64, 64, 256], stage=2, block='c')
	
	#Stage 3
	X = convolutional_block(X, f=3, filters=[128,128,512], stage=3, block='a', s=2)
	X = identify_block(X, 3,[256,256,256], 3, 'b')
	X = identify_block(X, 3,[256,256,256], 3, 'c')
	X = identify_block(X, 3,[256,256,256], 3, 'd')
	
	#Stage 4
	X = convolutional_block(X, f=3, filters=[256,256,1024], stage=4, block='a', s=2)
	X = identify_block(X, 3,[256,256,256], 4, 'b')
	X = identify_block(X, 3,[256,256,256], 4, 'c')
	X = identify_block(X, 3,[256,256,256], 4, 'd')
	X = identify_block(X, 3,[256,256,256], 4, 'e')
	X = identify_block(X, 3,[256,256,256], 4, 'f')
	
	#Stage 5
	X = convolutional_block(X, f=3, filters=[512,512,2048], stage=5, block='a', s=2)
	X = identify_block(X, f=3, filters=[512,512,2048], stage=5, block='b')
	X = identify_block(X, f=3, filters=[512,512,2048], stage=5, block='c')
	
	#AVGPOOL
	X = AveragePooling2D((2,2), name='avg_pool')(X)
	
	#Output layer
	X = Flatten()(X)
	X = Dense(classes, activation='softmax', name='fc'+str(classes),kernel_initializer=glorot_uniform(seed=0))(X)
	
	#Create model
	model = Model(input_shape=X_input, outputs=X, name='ResNet50')
	
	return model

In [12]:
model = ResNet50(input_shape=(64,64,3), classes=6)

ValueError: Operands could not be broadcast together with shapes (13, 13, 256) (15, 15, 256)